### **0. Getting ready**

#### **0.1. Importing necessary packages**

In [1]:
# for changing directory
import os

# for opening and working with the .grib/.nv files
import cfgrib
import cftime
import warnings
import xarray as xr
xr.set_options(display_style='html')
warnings.simplefilter("ignore")

# for calculations
import numpy as np
from sklearn.metrics import mean_squared_error
import xesmf as xe                             # for regridding
from scipy.interpolate import interp1d

from scipy.optimize import differential_evolution

from scipy import integrate

from scipy import stats                        # for regression statistics
from scipy.stats import linregress             # for linear regression
from scipy.stats import pearsonr               

# for nice displaying
import pandas as pd               
from IPython.display import HTML

# for nice plots
import intake
%matplotlib inline
from glob import glob
import matplotlib.pyplot as plt
import matplotlib.cm as cm                      # for the color map
import matplotlib.colors as mcolors

# for skiping running a certain cell
from IPython.core.magic import register_cell_magic

@register_cell_magic
def comment(line, cell):
    return

import dask
import dask.array as da


#### **0.2. Functions**

In [2]:
# Function to extract model name
def extract_model_name(string_name):
    return string_name.split('.')[2]

# Function for grid spacing
def grid(array):
    
    diff = []
    
    for i in range(1, len(array)):
                    
        diff_i = abs(array[i]-array[i-1])
                    
        diff.append(diff_i)
                    
    grid = np.mean(diff)
    
    return grid

# Function for extracting the number of the each year
def extract_year(time):
    
    # In case time is given in a cftime format    
    if isinstance(time, cftime.datetime):
        return str(time.year)
    
    # In case time is given in regular 'yyyy-mm-dd-hh-mm' format
    elif isinstance(time, np.datetime64):
        return str(pd.to_datetime(time).year)
        
    else:
        raise ValueError("Unsupported time format")

In [3]:
# Constants 

#Radius of the Earth in [m]:
R=6371000 

# Function for calculating the heat transport
def heat_transport(radiation, lat):
    
    integrad_1 = radiation 

    integrad_2 = (2*np.pi*R**2)*np.cos(np.deg2rad(lat))*(integrad_1) #no correction for the imbalance

    #Integration in rads:
    heat_tr = integrate.cumulative_trapezoid(integrad_2, x=np.deg2rad(lat), dx = np.deg2rad(2), initial=0)

    #[W] --> [PW]
    heat_tr = heat_tr*(10**-15)   
    
    return heat_tr

### **A. Data from pangeo**

I'm following [this tutorial](https://gallery.pangeo.io/repos/pangeo-gallery/cmip6/intake_ESM_example.html) of loading the CMIP6 data without downloading every single thing. 

#### **0.1. Getting the data**

In [4]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
col

,unique
activity_id,18
institution_id,36
source_id,88
experiment_id,170
member_id,657
table_id,37
variable_id,700
grid_label,10
zstore,514818
dcpp_init_year,60


In [5]:
# Selecting the parameters I need
# Same filters as in the CMIP6 page (https://esgf-data.dkrz.de/search/cmip6-dkrz/)

experiment_id = ['piControl', 'abrupt-4xCO2']

variable_id = ['rlut', 'rsdt', 'rsut', 'hfls', 'hfss', 'rlds', 'rlus', 'rsds', 'rsus'] # the last variable is the surface temperature, we'll use for the 
                                                                                              # temperature analysis in Sec.3.
source_id = ['CMCC-CM2-SR5', 'CanESM5', 'GISS-E2-1-G', 'GISS-E2-2-G', 'MIROC6', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'NorESM2-MM']
source_id = sorted(source_id)

#['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-CM-1-1-MR', 'BCC-CSM2-MR', 'BCC-ESM1', 'CESM2', 'CESM2-FV2', 'CESM2-WACCM', 'CESM2-WACCM-FV2', 'CMCC-CM2-SR5', 'CanESM5', 'E3SM-1-0', 'FGOALS-f3-L', 'FGOALS-g3', 'GFDL-ESM4', 'GISS-E2-1-G', 'GISS-E2-1-H', 'GISS-E2-2-G', 'IITM-ESM', 'INM-CM4-8', 'INM-CM5-0', 'MIROC6', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'NorCPM1', 'NorESM2-MM', 'SAM0-UNICON', 'TaiESM1']
#that is all the models that have all 9 heat fluxes. However, only few of them have hfbasin as well. For economy purposes, we will work only with the common models. 

In [6]:
# Sorting the data alphabetically, making sure we've got what we need

cat1 = col.search(experiment_id = experiment_id, table_id = 'Amon', variable_id = variable_id, activity_id = 'CMIP', member_id = 'r1i1p1f1', source_id = source_id) 

data = cat1.df

df_sorted = data.sort_values(by=[data.columns[2], data.columns[3]], ascending=True)

# Reset index
df_sorted.reset_index(drop=True, inplace=True)

HTML(df_sorted.to_html())

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,CMCC,CMCC-CM2-SR5,abrupt-4xCO2,r1i1p1f1,Amon,rlut,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/abrupt-4xCO2/r1i1p1f1/Amon/rlut/gn/v20200616/,NaN,20200616
1,CMIP,CMCC,CMCC-CM2-SR5,abrupt-4xCO2,r1i1p1f1,Amon,rsds,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/abrupt-4xCO2/r1i1p1f1/Amon/rsds/gn/v20200616/,NaN,20200616
2,CMIP,CMCC,CMCC-CM2-SR5,abrupt-4xCO2,r1i1p1f1,Amon,rsut,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/abrupt-4xCO2/r1i1p1f1/Amon/rsut/gn/v20200616/,NaN,20200616
3,CMIP,CMCC,CMCC-CM2-SR5,abrupt-4xCO2,r1i1p1f1,Amon,rsdt,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/abrupt-4xCO2/r1i1p1f1/Amon/rsdt/gn/v20200616/,NaN,20200616
4,CMIP,CMCC,CMCC-CM2-SR5,abrupt-4xCO2,r1i1p1f1,Amon,rsus,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/abrupt-4xCO2/r1i1p1f1/Amon/rsus/gn/v20200616/,NaN,20200616
5,CMIP,CMCC,CMCC-CM2-SR5,abrupt-4xCO2,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/abrupt-4xCO2/r1i1p1f1/Amon/rlds/gn/v20200616/,NaN,20200616
6,CMIP,CMCC,CMCC-CM2-SR5,abrupt-4xCO2,r1i1p1f1,Amon,hfls,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/abrupt-4xCO2/r1i1p1f1/Amon/hfls/gn/v20200616/,NaN,20200616
7,CMIP,CMCC,CMCC-CM2-SR5,abrupt-4xCO2,r1i1p1f1,Amon,hfss,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/abrupt-4xCO2/r1i1p1f1/Amon/hfss/gn/v20200616/,NaN,20200616
8,CMIP,CMCC,CMCC-CM2-SR5,abrupt-4xCO2,r1i1p1f1,Amon,rlus,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/abrupt-4xCO2/r1i1p1f1/Amon/rlus/gn/v20200616/,NaN,20200616
9,CMIP,CMCC,CMCC-CM2-SR5,piControl,r1i1p1f1,Amon,hfls,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/piControl/r1i1p1f1/Amon/hfls/gn/v20200616/,NaN,20200616


In [7]:
# Loading the data, putting everything into a list

dset_dict = cat1.to_dataset_dict(zarr_kwargs={'consolidated': True})
models_experiments_list = list(dset_dict.keys())


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


#### **0.2. Useful information about our data: Grid and Time intervals**

In [8]:
# Model names
models_experiments = []

for dataset in models_experiments_list:
    names = dataset.split('.')    
    dataset_name = names[2] + '.' + names[3]        
    models_experiments.append(dataset_name)

In [9]:
# Grid
grid_lon = []
grid_lat = []
time_intervals = []
lats = []

for dataset in models_experiments_list:    
    dataset_name = extract_model_name(dataset)
    
    lon = dset_dict[dataset]['lon'].values
    lat = dset_dict[dataset]['lat'].values

    lats.append(lat)    
    grid_lon.append(grid(lon))
    grid_lat.append(grid(lat)) 

In [10]:
# Time
# Getting the time values

for dataset in models_experiments_list:
        
    time = dset_dict[dataset]['time'].values    
    time_intervals.append(time)   
    
# Keeping only the first and the last year

begin = []
end = []

for i in range (0, len(time_intervals)):
    
    begin.append(time_intervals[i][0])    
    end.append(time_intervals[i][-1])
    
# Extract years
begin_years = [extract_year(b) for b in begin]
end_years = [extract_year(e) for e in end]

time_periods = []

for i, j in zip(begin_years, end_years):

    time_period = int(j)-int(i)+1

    time_periods.append(time_period)    

In [11]:
# Print everything in a nice way

# Create DataFrame
df_grid_time = pd.DataFrame({
    'Model': models_experiments,
    'Longitude grid': grid_lon,
    'Latitude grid': grid_lat,
    'Begin year': begin_years,
    'End year': end_years,
    'Time period': time_periods
})

# Sort Alphabetically
df_grid_time_sorted = df_grid_time.sort_values(by=['Model'], ascending=True)

# Reset index
df_grid_time_sorted.reset_index(drop=True, inplace=True)

# Display DataFrame
HTML(df_grid_time_sorted.to_html())

,Model,Longitude grid,Latitude grid,Begin year,End year,Time period
0,CMCC-CM2-SR5.abrupt-4xCO2,1.25000,0.942408,1850,2014,165
1,CMCC-CM2-SR5.piControl,1.25000,0.942408,1850,2349,500
2,CanESM5.abrupt-4xCO2,2.81250,2.789327,1850,2000,151
3,CanESM5.piControl,2.81250,2.789327,5201,6200,1000
4,GISS-E2-1-G.abrupt-4xCO2,2.50000,2.000000,1850,2000,151
5,GISS-E2-1-G.piControl,2.50000,2.000000,4150,5000,851
6,GISS-E2-2-G.abrupt-4xCO2,2.50000,2.000000,1850,2000,151
7,GISS-E2-2-G.piControl,2.50000,2.000000,2000,2150,151
8,MIROC6.abrupt-4xCO2,1.40625,1.400437,3200,3449,250
9,MIROC6.piControl,1.40625,1.400437,3200,3999,800


#### **0.3. Processing**

(Separating the datasets in piControl, x4CO2 lists and sorting them in alphabetical order)

In [12]:
ds_piControl = []
ds_x4CO2 = []

# Extracting the names of models from the dataset names

piControl_names = [name for name in models_experiments_list if 'piControl' in name]
abrupt4xCO2_names = [name for name in models_experiments_list if 'abrupt-4xCO2' in name]

# Now, we can sort the lists alphabetically by the MODEL name and not just the dataframe name

piControl_names.sort(key=extract_model_name)
abrupt4xCO2_names.sort(key=extract_model_name)

# Dropping everything in the correct list

for i, j in zip(piControl_names, abrupt4xCO2_names):
    
    ds_piControl.append(dset_dict[i])
    ds_x4CO2.append(dset_dict[j])

#### **1. Heat Fluxes**

# Merged script

#VERY CAREFUL NOW!!! The new lists are organized like the esgf downloaded data: model-> experiment#Also: it might be too heavy to run, so it might crush. 
#in that case, it's better to have 2 separate scripts
#the reson I insist in one script is that then we can have everythiung organised with models in alphabetical order so it's easier to match the correc ocean basins without any m istakes.

In [13]:
# Lists to loop over. 

models_pangeo = source_id
models_esgf = ['CanESM5-1', 'CIESM', 'CMCC-ESM2', 'EC-Earth3-AerChem', 'EC-Earth3-CC', 'EC-Earth3-Veg', 'EC-Earth3-Veg-LR', 'IPSL-CM6A-LR', 'IPSL-CM6A-MR1', 'MRI-ESM2-0', 'NorESM2-LM']
models_esgf = sorted(models_esgf)
#Same as before, we only work with the models that also include hfbasin.
#Original list with the models that contain all 9 heat fluxes:['CanESM5-1', 'CAS-ESM2-0', 'CIESM', 'CMCC-ESM2', 'E3SM-2-0', 'EC-Earth3-AerChem', 'EC-Earth3-CC', 'EC-Earth3-Veg', 'EC-Earth3-Veg-LR', 'GISS-E2-2-H', 'IPSL-CM6A-LR', 'IPSL-CM6A-MR1', 'MPI-ESM-1-2-HAM', 'MRI-ESM2-0', 'NESM3', 'NorESM2-LM']

models_all = sorted(models_pangeo + models_esgf) #alphabetical order

experiments_pangeo = [ds_piControl, ds_x4CO2]
experiments_esgf = ['piControl', 'x4CO2']

# All of the atmospheric heat fluxes
components = variable_id

In [14]:
def time_ave_heat_trs (heat_fluxes, lon, lat):

    toa_rad = heat_fluxes[1] - heat_fluxes[0] - heat_fluxes[2]
    atm_rad = toa_rad + heat_fluxes[3] + heat_fluxes[4] - heat_fluxes[5] + heat_fluxes[6] - heat_fluxes[7] + heat_fluxes[8]

    toa_zonal_ave = (np.trapz(toa_rad, x=np.deg2rad(lon), axis=2)) / (2 * np.pi)
    atm_zonal_ave = (np.trapz(atm_rad, x=np.deg2rad(lon), axis=2)) / (2 * np.pi) 

    # Taking the time average #############################################################################

    toa_time_ave = toa_zonal_ave.mean(axis=0)
    atm_time_ave = atm_zonal_ave.mean(axis=0)

    toa_ht = heat_transport(toa_time_ave, lat)
    atm_ht = heat_transport(atm_time_ave, lat)

    return toa_ht, atm_ht

In [15]:
def time_depend_heat_trs (heat_fluxes, lon, lat, num_y):

    toa_rad = heat_fluxes[1] - heat_fluxes[0] - heat_fluxes[2]
    atm_rad = toa_rad + heat_fluxes[3] + heat_fluxes[4] - heat_fluxes[5] + heat_fluxes[6] - heat_fluxes[7] + heat_fluxes[8]

    toa_zonal_ave = (np.trapz(toa_rad, x=np.deg2rad(lon), axis=2)) / (2 * np.pi)
    atm_zonal_ave = (np.trapz(atm_rad, x=np.deg2rad(lon), axis=2)) / (2 * np.pi) 

    # Not taking the time average (you have to comment out the previous 4 lines) ##########################

    # Calculating the monthly means so we have yearly data

    toa_zonal_ave_reshaped = toa_zonal_ave.reshape(num_y, 12, len(lat))
    toa_zonal_ave = np.mean(toa_zonal_ave_reshaped, axis=1)

    atm_zonal_ave_reshaped = atm_zonal_ave.reshape(num_y, 12, len(lat))
    atm_zonal_ave = np.mean(atm_zonal_ave_reshaped, axis=1)
        
    toa_ht_t = []
    atm_ht_t = []

    for i in range(num_y):
            
        toa_ht_t.append(heat_transport(toa_zonal_ave[i, :], lat))
        atm_ht_t.append(heat_transport(atm_zonal_ave[i, :], lat))

    toa_ht_t = np.array(toa_ht_t)
    atm_ht_t = np.array(atm_ht_t)

    return toa_ht_t, atm_ht_t

In [16]:
# Choosing the time period

# years 0-150:

#start_y = 0
#end_y = 1800

#period = 1789

#start_number = 0 # for plot labeling
#end_number = 150

# years 50-150:

#start_y = 600
#end_y = 1800

#period = 1189 #(1200-12+1)

#start_number = 50 # for plot labeling
#end_number = 150

# years 100-150

#start_y = 1200
#end_y = 1800

#period = 589 #(600-12+1)

#start_number = 100 # for plot labeling
#end_number = 150

# every 30 years

start_y = 1080
end_y = 1800

In [17]:
# Do you want time average or not? True: Yes plese, False: Not thank you

time_average = False

num_y = 60 # the number of years I'm working with when I'm not using time average

# Initialize dictionaries to hold the component arrays for each model
toa_ht_model = []
atm_ht_model = []

lat_model = []

In [18]:
stop

NameError: name 'stop' is not defined

In [19]:
for model in models_all:
        
    if model in models_pangeo: #already alphabetically ordered, so no problem in the big alphabetical list
        
        model_idx = models_pangeo.index(model)
        
        toa_ht_exp = []
        atm_ht_exp = []
        
        lat_exp = []

        for experiment in experiments_pangeo:
            # Initialize dictionaries to hold the component arrays for each model

            model_exp = experiment[model_idx]
            
            string_name = model_exp.attrs.get('intake_esm_dataset_key')
            model_name = extract_model_name(string_name)
            print(model_name, ' is a pangeo model')
        
            # Select 50-year slice for performance
            ds_50 = model_exp.isel(time=slice(start_y, end_y))
        
            lon = ds_50['lon'].values                
            lat = ds_50['lat'].values  
                
            heat_fluxes = []
        
            for component in components:
                #print(component)
        
                heat_fluxes.append(ds_50[component][0, 0, :, :, :].values)
                
            # Time averaged heat transports 
            if time_average == True:                
                toa_ht, atm_ht = time_ave_heat_trs (heat_fluxes, lon, lat)

            # Non time-averaged heat transports
            else:
                toa_ht, atm_ht = time_depend_heat_trs (heat_fluxes, lon, lat, num_y)
            
            toa_ht_exp.append(toa_ht)
            atm_ht_exp.append(atm_ht)
            lat_exp.append(lat)

        toa_ht_model.append(toa_ht_exp)
        atm_ht_model.append(atm_ht_exp)
        lat_model.append(lat_exp)

    elif model in models_esgf:

        toa_ht_exp = []
        atm_ht_exp = []
        
        lat_exp = []

        for experiment in experiments_esgf:
            print(model, ' is an esgf model')
            heat_fluxes = []
            
            for component in components:
                #print(component)
    
                folder_path = f'/mn/vann/chrikap/CMIP6/{model}/{experiment}/ATM/{component}'
                file_names = sorted(os.listdir(folder_path))
    
                # Using dask to read datasets
                datasets = [xr.open_dataset(os.path.join(folder_path, file_name), chunks={'time': 10}) for file_name in file_names if file_name.endswith('.nc')]
    
                if len(datasets) == 1:
                    ds = datasets[0]
                    ds_50 = ds.isel(time=slice(start_y, end_y))
                else:
                    # Merge all the datasets in the list along the same dimension
                    merged_datasets = xr.concat(datasets, dim='time')
                    ds_50 = merged_datasets.isel(time=slice(start_y, end_y))
    
                heat_fluxes.append(ds_50[component].values)

            lon = ds_50['lon'].values     
            lat = ds_50['lat'].values     

            # Time averaged heat transports 
            if time_average == True:                
                toa_ht, atm_ht = time_ave_heat_trs (heat_fluxes, lon, lat)

            # Non time-averaged heat transports
            else:
                toa_ht, atm_ht = time_depend_heat_trs (heat_fluxes, lon, lat, num_y)
            
            toa_ht_exp.append(toa_ht)
            atm_ht_exp.append(atm_ht)
            lat_exp.append(lat)

        toa_ht_model.append(toa_ht_exp)
        atm_ht_model.append(atm_ht_exp)
        lat_model.append(lat_exp)

CIESM  is an esgf model
CIESM  is an esgf model
CMCC-CM2-SR5  is a pangeo model
CMCC-CM2-SR5  is a pangeo model
CMCC-ESM2  is an esgf model
CMCC-ESM2  is an esgf model
CanESM5  is a pangeo model
CanESM5  is a pangeo model
CanESM5-1  is an esgf model
CanESM5-1  is an esgf model
EC-Earth3-AerChem  is an esgf model
EC-Earth3-AerChem  is an esgf model
EC-Earth3-CC  is an esgf model
EC-Earth3-CC  is an esgf model
EC-Earth3-Veg  is an esgf model
EC-Earth3-Veg  is an esgf model
EC-Earth3-Veg-LR  is an esgf model
EC-Earth3-Veg-LR  is an esgf model
GISS-E2-1-G  is a pangeo model
GISS-E2-1-G  is a pangeo model
GISS-E2-2-G  is a pangeo model
GISS-E2-2-G  is a pangeo model
IPSL-CM6A-LR  is an esgf model
IPSL-CM6A-LR  is an esgf model
IPSL-CM6A-MR1  is an esgf model
IPSL-CM6A-MR1  is an esgf model
MIROC6  is a pangeo model
MIROC6  is a pangeo model
MPI-ESM1-2-HR  is a pangeo model
MPI-ESM1-2-HR  is a pangeo model
MPI-ESM1-2-LR  is a pangeo model
MPI-ESM1-2-LR  is a pangeo model
MRI-ESM2-0  is an es

In [21]:
# Regridding the curves along latitude

target_lat = np.arange(-90, 92, 2)  # Define the target latitude grid

In [ ]:
%%comment
# Time averaged values
toa_int_model = []
atm_int_model = []

for toa_exp, atm_exp, lat_exp in zip(toa_ht_model, atm_ht_model, lat_model):

    toa_int_exp = []
    atm_int_exp = []
    
    for toa, atm, lat_v in zip(toa_exp, atm_exp, lat_exp):

        old_lat = np.array(lat_v)

        toa_interp_func = interp1d(old_lat, toa, kind='linear', bounds_error=False, fill_value='extrapolate')
        atm_interp_func = interp1d(old_lat, atm, kind='linear', bounds_error=False, fill_value='extrapolate')

        toa_ht_interp = toa_interp_func(target_lat)
        atm_ht_interp = atm_interp_func(target_lat)

        toa_int_exp.append(toa_ht_interp)
        atm_int_exp.append(atm_ht_interp)

    toa_int_model.append(toa_int_exp)
    atm_int_model.append(atm_int_exp)

toa_int_model = np.array(toa_int_model)
atm_int_model = np.array(atm_int_model)

In [ ]:
%comment
# Storing everything because re-running the script takes a lot of time

# Time averages
os.chdir('/home/chrikap/Desktop/atmospheric heat transport/heat transport datasets/Annual_Variations/CMIP6/Paper_Scripts/heat_transports')

np.save('toa_int_model', toa_int_model)
np.save('atm_int_model', atm_int_model)

In [ ]:
%comment

os.chdir('/home/chrikap/Desktop/atmospheric heat transport/heat transport datasets/Annual_Variations/CMIP6/Paper_Scripts/heat_transports')
toa_int_model = np.load('toa_int_model.npy')
atm_int_model = np.load('atm_int_model.npy')

In [22]:
# No time average

# Time averaged values
toa_int_model_t = []
atm_int_model_t = []

for toa_exp, atm_exp, lat_exp in zip(toa_ht_model, atm_ht_model, lat_model):

    toa_int_exp = []
    atm_int_exp = []
    
    for toa, atm, lat_v in zip(toa_exp, atm_exp, lat_exp):
        
        toa_int_time = []
        atm_int_time = []

        old_lat = np.array(lat_v)

        for toa_time, atm_time in zip(toa, atm):

            toa_interp_func = interp1d(old_lat, toa_time, kind='linear', bounds_error=False, fill_value='extrapolate')
            atm_interp_func = interp1d(old_lat, atm_time, kind='linear', bounds_error=False, fill_value='extrapolate')
    
            toa_ht_interp = toa_interp_func(target_lat)
            atm_ht_interp = atm_interp_func(target_lat)
    
            toa_int_time.append(toa_ht_interp)
            atm_int_time.append(atm_ht_interp)
            
        toa_int_exp.append(toa_int_time)
        atm_int_exp.append(atm_int_time)            

    toa_int_model_t.append(toa_int_exp)
    atm_int_model_t.append(atm_int_exp)

toa_int_model_t = np.array(toa_int_model_t)
atm_int_model_t = np.array(atm_int_model_t)

In [23]:
toa_int_model_t.shape

(19, 2, 60, 91)

In [24]:
# Storing everything because re-running the script takes a lot of time

# No time averages
os.chdir('/home/chrikap/Desktop/atmospheric heat transport/heat transport datasets/Annual_Variations/CMIP6/Paper_Scripts/heat_transports')

#np.save('toa_int_model_0_30', toa_int_model_t)
#np.save('atm_int_model_0_30', atm_int_model_t)

#np.save('toa_int_model_30_60', toa_int_model_t)
#np.save('atm_int_model_30_60', atm_int_model_t)

#np.save('toa_int_model_60_90', toa_int_model_t)
#np.save('atm_int_model_60_90', atm_int_model_t)

np.save('toa_int_model_90_150', toa_int_model_t)
np.save('atm_int_model_90_150', atm_int_model_t)

In [25]:
os.chdir('/home/chrikap/Desktop/atmospheric heat transport/heat transport datasets/Annual_Variations/CMIP6/Paper_Scripts/heat_transports')
toa_int_model_0_30 = np.load('toa_int_model_0_30.npy')
atm_int_model_0_30 = np.load('atm_int_model_0_30.npy')

toa_int_model_30_60 = np.load('toa_int_model_30_60.npy')
atm_int_model_30_60 = np.load('atm_int_model_30_60.npy')

toa_int_model_60_90 = np.load('toa_int_model_60_90.npy')
atm_int_model_60_90 = np.load('atm_int_model_60_90.npy')

toa_int_model_90_150 = np.load('toa_int_model_90_150.npy')
atm_int_model_90_150 = np.load('atm_int_model_90_150.npy')

In [28]:
toa_int_model = np.concatenate([toa_int_model_0_30,
                                toa_int_model_30_60,
                                toa_int_model_60_90,
                                toa_int_model_90_150], axis = 2)

atm_int_model = np.concatenate([atm_int_model_0_30,
                               atm_int_model_30_60,
                               atm_int_model_60_90,
                               atm_int_model_90_150], axis = 2)